# Projet Fraude 



In [50]:
#importer vos lib
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from datetime import datetime
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import DistanceMetric
from time import perf_counter 
import warnings
warnings.simplefilter(action='ignore')

dFraud = pd.read_csv('./DATA/Fraud_Data.csv')
dIp = pd.read_csv('./DATA/IpAddress_to_Country.csv')


In [33]:
dFraud.head()


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,device_num,signup_day,signup_week,purchase_day,purchase_week,ip_num,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,1,1,9,5,16,1,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,1,6,23,0,24,1,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,12,3,1,3,1,12,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,1,1,18,0,19,1,NULL
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,1,1,30,2,37,1,United States


In [36]:
dIp.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [35]:
df = dFraud

def lookup_country(ip):
    result =  dIp[(dIp['lower_bound_ip_address']<ip) & (dIp['upper_bound_ip_address']>=ip)]['country']
    if len(result) :
        return result.tolist()[0]
    else:
        return 'NULL'

df['country'] = df.apply(lambda row: lookup_country(row['ip_address']), axis=1)

df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,device_num,signup_day,signup_week,purchase_day,purchase_week,ip_num,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,1,1,9,5,16,1,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,1,6,23,0,24,1,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,12,3,1,3,1,12,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,1,1,18,0,19,1,NULL
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,1,1,30,2,37,1,United States


### Importer les dataset

In [41]:
purchase = df['purchase_time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
signup = df['signup_time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['time_diff'] = purchase - signup
df['time_diff'] = df['time_diff'].apply(lambda x: x.days)
df

TypeError: strptime() argument 1 must be str, not Timestamp

In [42]:
ip_address = dFraud[['user_id', 'ip_address']].groupby('ip_address').count().reset_index()
ip_address= ip_address.rename(columns={'user_id': 'ip_num'})
dFraud = dFraud.merge(ip_address, how='left', on='ip_address')
dFraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,device_num,signup_day,signup_week,purchase_day,purchase_week,ip_num_x,country,ip_num_y
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,1,1,9,5,16,1,Japan,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,1,6,23,0,24,1,United States,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,12,3,1,3,1,12,United States,12
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,1,1,18,0,19,1,NULL,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,1,1,30,2,37,1,United States,1


In [43]:
device_num = dFraud[['user_id', 'device_id']].groupby('device_id').count().reset_index()
device_num = device_num.rename(columns={'user_id': 'device_num'})
dFraud = dFraud.merge(device_num, how='left', on='device_id')
dFraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,device_num_x,signup_day,signup_week,purchase_day,purchase_week,ip_num_x,country,ip_num_y,device_num_y
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,1,1,9,5,16,1,Japan,1,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,1,6,23,0,24,1,United States,1,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,12,3,1,3,1,12,United States,12,12
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,1,1,18,0,19,1,NULL,1,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,1,1,30,2,37,1,United States,1,1


In [44]:
dFraud['signup_time'] = pd.to_datetime(dFraud['signup_time'])
dFraud['signup_day'] = dFraud['signup_time'].dt.dayofweek
dFraud['signup_week'] = dFraud['signup_time'].dt.week 
dFraud['purchase_time'] = pd.to_datetime(dFraud['purchase_time'])
dFraud['purchase_day'] = dFraud['purchase_time'].dt.dayofweek
dFraud['purchase_week'] = dFraud['purchase_time'].dt.week 
dFraud

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,device_num_x,signup_day,signup_week,purchase_day,purchase_week,ip_num_x,country,ip_num_y,device_num_y
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,1,1,9,5,16,1,Japan,1,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,1,6,23,0,24,1,United States,1,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,12,3,1,3,1,12,United States,12,12
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,1,1,18,0,19,1,NULL,1,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,1,1,30,2,37,1,United States,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,XPSKTWGPWINLR,SEO,Chrome,M,28,3.451155e+09,1,2,1,5,6,13,1,United States,1,2
151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2.439047e+09,0,1,4,20,1,22,1,Netherlands,1,1
151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2.748471e+09,0,1,1,10,2,21,1,Japan,1,1
151110,207709,2015-07-09 20:06:07,2015-09-07 09:34:46,46,CMCXFGRHYSTVJ,SEO,Chrome,M,37,3.601175e+09,0,2,3,28,0,37,1,United States,1,2


In [45]:
y = dFraud['class']
data = dFraud.drop(['user_id', 'signup_time', 'purchase_time', 'class','device_id'], axis = 1)
print(dFraud)
X = data
X.isnull().sum()

        user_id         signup_time       purchase_time  purchase_value  \
0         22058 2015-02-24 22:55:49 2015-04-18 02:47:11              34   
1        333320 2015-06-07 20:39:50 2015-06-08 01:38:54              16   
2          1359 2015-01-01 18:52:44 2015-01-01 18:52:45              15   
3        150084 2015-04-28 21:13:25 2015-05-04 13:54:50              44   
4        221365 2015-07-21 07:09:52 2015-09-09 18:40:53              39   
...         ...                 ...                 ...             ...   
151107   345170 2015-01-27 03:03:34 2015-03-29 00:30:47              43   
151108   274471 2015-05-15 17:43:29 2015-05-26 12:24:39              35   
151109   368416 2015-03-03 23:07:31 2015-05-20 07:07:47              40   
151110   207709 2015-07-09 20:06:07 2015-09-07 09:34:46              46   
151111   138208 2015-06-10 07:02:20 2015-07-21 02:03:53              20   

            device_id  source browser sex  age    ip_address  class  \
0       QVPSPJUOCKZAR     SE

purchase_value    0
source            0
browser           0
sex               0
age               0
ip_address        0
device_num_x      0
signup_day        0
signup_week       0
purchase_day      0
purchase_week     0
ip_num_x          0
country           0
ip_num_y          0
device_num_y      0
dtype: int64

In [46]:
lb = preprocessing.LabelEncoder()
X['source'] = lb.fit_transform(X['source'])
X['browser'] = lb.fit_transform(X['browser'])
X['sex'] = lb.fit_transform(X['sex'])
X['country'] = lb.fit_transform(X['country'])

X.head()

,purchase_value,source,browser,sex,age,ip_address,device_num_x,signup_day,signup_week,purchase_day,purchase_week,ip_num_x,country,ip_num_y,device_num_y
0,34,2,0,1,39,7.327584e+08,1,1,9,5,16,1,84,1,1
1,16,0,0,0,53,3.503114e+08,1,6,23,0,24,1,172,1,1
2,15,2,3,1,53,2.621474e+09,12,3,1,3,1,12,172,12,12
3,44,2,4,1,41,3.840542e+09,1,1,18,0,19,1,115,1,1
4,39,0,4,1,45,4.155831e+08,1,1,30,2,37,1,172,1,1


In [47]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [48]:
cls_forest = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', max_depth = 50 , min_samples_leaf = 3, min_samples_split = 3, oob_score = True  )
cls_forest.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=50, min_samples_leaf=3,
                       min_samples_split=3, n_estimators=20, oob_score=True)

In [51]:
predi = cls_forest.predict(X_test)
predi
print(classification_report(y_test, predi))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98     34227
           1       0.99      0.54      0.70      3551

    accuracy                           0.96     37778
   macro avg       0.97      0.77      0.84     37778
weighted avg       0.96      0.96      0.95     37778



In [53]:
imp = cls_forest.feature_importances_
imp 

array([0.05853894, 0.01374369, 0.01750346, 0.00922053, 0.05085101,
       0.0716962 , 0.11246527, 0.03397151, 0.09319015, 0.02589413,
       0.15742744, 0.08750039, 0.03249162, 0.13283279, 0.10267289])

In [54]:
data.columns.values[:-1]

array(['purchase_value', 'source', 'browser', 'sex', 'age', 'ip_address',
       'device_num_x', 'signup_day', 'signup_week', 'purchase_day',
       'purchase_week', 'ip_num_x', 'country', 'ip_num_y'], dtype=object)

In [57]:
imp = pd.DataFrame(cls_forest.feature_importances_,index = X_train.columns,columns=['importance']).sort_values('importance',ascending=False)

sns.catplot(x=imp['importance'], y=imp['importance'].index,kind="bar", data=imp, height=10)

NameError: name 'sns' is not defined